### Download and make the dataset ready in Kaggle 


In [18]:

# ## uncomment if The zip file of the dataset isn't downloaded,extraced 
# !pip install gdown
# Copy the link. The file ID is the long string of characters between d/ and /view.
#For example, in the URL https://drive.google.com/file/d/1aBcDeFgHiJkLmNoPqRsTuVwXyZ/view?usp=sharing, 
#the file ID is 1aBcDeFgHiJkLmNoPqRsTuVwXyZ
# !mkdir /kaggle/tmp
# !gdown  1pzXpA5Cz0DJmjRsLxlqRNnJq-kOUvojb -O /kaggle/tmp/Labeled_CICMODBUS2023.zip
# !unzip /kaggle/tmp/Labeled_CICMODBUS2023.zip -d /kaggle/working/

# # ## uncomment if the python modules (modbus.py,utils.py ,...) not cloned  and added to the path 

# !git clone https://github.com/hamid-rd/FLBased-ICS-NIDS.git
# import sys
# # Add the repository folder to the Python path
# repo_path = '/kaggle/working/FLBased-ICS-NIDS'
# repo_input_path = '/kaggle/input/training/FLBased-ICS-NIDS'
# dataset_path = '/kaggle/input/training/'

# for path in {repo_path,repo_input_path,dataset_path}:
#     if path not in sys.path:
#         sys.path.append(path)


Downloading...
From (original): https://drive.google.com/uc?id=1pzXpA5Cz0DJmjRsLxlqRNnJq-kOUvojb
From (redirected): https://drive.google.com/uc?id=1pzXpA5Cz0DJmjRsLxlqRNnJq-kOUvojb&confirm=t&uuid=d8e1c4d9-f327-45b8-bd78-ecb863cd1471
To: /kaggle/tmp/Labeled_CICMODBUS2023.zip
100%|███████████████████████████████████████| 12.3G/12.3G [01:34<00:00, 130MB/s]
Archive:  /kaggle/tmp/Labeled_CICMODBUS2023.zip
 extracting: /kaggle/working/ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-25-labeled.csv  
 extracting: /kaggle/working/ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-30-labeled.csv  
 extracting: /kaggle/working/ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-26-labeled.csv  
 extracting: /kaggle/working/ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-17-labeled.csv  
 extracting: /kaggle/working/ModbusDataset/benign/network-wide-pcap-

In [19]:
# To test if every thing is okay (modbus.py class and correct number of founded csv files )
from modbus import ModbusDataset,ModbusFlowStream

# dataset_directory = "/kaggle/working/ModbusDataset" 
dataset_directory = "/kaggle/input/training/ModbusDataset" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

# Don't forget to save version in kaggle (to save outputs written on the disk (/kaggle/working/))  

 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

### Unsupervised GRU-VAE training  

In [20]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np # For standard deviation calculation
from modbus import ModbusDataset,ModbusFlowStream
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,recall_score
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from utils import load_scalers
from random import SystemRandom
from sklearn.model_selection import train_test_split
import itertools
import torch.nn.init as init
# from pythresh.thresholds.zscore import ZSCORE
# from pythresh.thresholds.mad import MAD
# from pythresh.thresholds.iqr import IQR
# from pythresh.thresholds.aucp import AUCP

# def compute_threshold(mse_values,k=1):
#     # """
#     # K-SIGMA
#     # Computes the anomaly detection threshold (for marking sample as Intrusion if the IS was greater )
#     # based on the mean and standard deviation of Mean Squared Error (MSE) values.
#     # Formula: thr = mean(MSE) + std(MSE)

#     # Args:
#     #     mse_values (torch.Tensor or list/np.array): A tensor or list of MSE values
#     #                                                 obtained from the validation set.

#     # Returns:
#     #     float: The calculated threshold.
#     # """
      ## example usage :
#       a=compute_threshold([1,2,3])
        # for (k,v) in a.items() :
            # print(k,v)
#     """
#     Computes anomaly detection thresholds using four different methods from pythresh.

#     Args:
#         mse_values (torch.Tensor or list/np.array): A tensor, list, or numpy array of 
#                                                        MSE values from the validation set.

#     Returns:
#         dict: A dictionary containing the calculated thresholds for each method.
#               Returns an empty dictionary if the input is empty.
#     """

#     if not isinstance(mse_values, torch.Tensor):
#         mse_values = torch.tensor(mse_values, dtype=torch.float32)
#     if isinstance(mse_values, torch.Tensor):
#         mse_np = mse_values.detach().cpu().numpy()
#     else:
#         mse_np = np.array(mse_values, dtype=np.float32)
#     if mse_values.numel() == 0:
#         return 0.0 
    
#     zscore_thresholder = ZSCORE()
#     mad_thresholder = MAD()
#     iqr_thresholder = IQR()
#     aucp_thresholder = AUCP()
    
#     # 2. Calculate the threshold from each method

#     mean_mse = torch.mean(mse_values)
#     std_mse = torch.std(mse_values)
#     # print("-----------",mean_mse,std_mse)
#     # print(sorted(list((mse_values)))[-10:-1])
#     k_sigma_threshold = mean_mse + k*std_mse
#     thresholds = {
#         'ZSCORE': zscore_thresholder.fit(mse_np).thresh_,
#         'MAD': mad_thresholder.fit(mse_np).thresh_,
#         'IQR': iqr_thresholder.fit(mse_np).thresh_,
#         'AUCP': aucp_thresholder.fit(mse_np).thresh_,
#         'K-SIGMA' :k_sigma_threshold.cpu().detach().numpy()
#     }
#     return thresholds
#     # return threshold.item() 

def compute_threshold(mse_values,k=1):

    """
    K-SIGMA
    Computes the anomaly detection threshold (for marking sample as Intrusion if the IS was greater )
    based on the mean and standard deviation of Mean Squared Error (MSE) values.
    Formula: thr = mean(MSE) + std(MSE)
    Args:
    mse_values (torch.Tensor or list/np.array): A tensor or list of MSE values

                            obtained from the validation set.
    Returns:
    float: The calculated threshold.

    """
    if not isinstance(mse_values, torch.Tensor):
        mse_values = torch.tensor(mse_values, dtype=torch.float32)
    if mse_values.numel() == 0:
        return 0.0
    mean_mse = torch.mean(mse_values)
    std_mse = torch.std(mse_values)
    print("-----------",mean_mse,std_mse)
    print(sorted(list((mse_values)))[-10:-1])
    threshold = mean_mse + k*std_mse
    return threshold.item()

def vae_loss_function(recon_x, x, mu, logvar,beta =1):
    """
    VAE loss function.
    """
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return (BCE + beta*KLD)


In [11]:
# import numpy as np
# # train the KNN detector
# from pyod.models.knn import KNN
# from pythresh.thresholds.clust import CLUST
# from pythresh.thresholds.comb import COMB
# from sklearn.dummy import DummyClassifier

# def my_custom_loss_func(y_true, y_pred):
#     diff = np.abs(y_true - y_pred).max()
#     return float(np.log1p(diff))
# # score will negate the return value of my_custom_loss_func,
# # which will be np.log(2), 0.693, given the values for X
# # and y defined below.
# X = [[1], [0],[2], [10],[1], [1],[1], [1],[1], [1]]
# y = [0, 1,1,1,1,1,1,1,0,0]

# clf = KNN()
# clf.fit(X)

# # get outlier likelihood scores
# decision_scores = clf.decision_scores_
# print(decision_scores)
# # # get outlier labels
# thres = COMB(thresholders = [CLUST(method='bgm', random_state=1234),
# CLUST(method='bgm', random_state=42),
# CLUST(method='bgm', random_state=9685),
# CLUST(method='bgm', random_state=111222)])
# thres = CLUST
# thres.fit(decision_scores)
# print(thres.thresh_)
# labels = thres.labels_ # or thres.predict(decision_scores)
# print(labels)

In [21]:
dataset_directory = "/kaggle/input/training/ModbusDataset" # change this to the folder contain benign and attack subdirs
modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

In [22]:
csv_files=[col for col in modbus.dataset["benign_dataset_dir"] if col.find("network-wide")!=-1]
sys_rand = SystemRandom()
sys_rand.shuffle(csv_files)
train_files,_ = train_test_split(csv_files,test_size=0.2,shuffle=True)
val_files = [col for col in modbus.dataset["benign_dataset_dir"] if col.find("ied1b")!=-1]
test_files=[col for col in modbus.dataset["attack_dataset_dir"]["compromised-scada"] if col.find("ied1b")!=-1][0:4]
print("ied1b comp ied attack ->\n test: ",len(test_files),test_files)
print("----------- Network-wide number of csv files -> \n ----------- train :",len(train_files),train_files,"\n -------- valid:",len(val_files),val_files)

ied1b comp ied attack ->
 test:  4 ['/kaggle/input/training/ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv', '/kaggle/input/training/ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-0-labeled.csv', '/kaggle/input/training/ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-1-labeled.csv', '/kaggle/input/training/ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-7-labeled.csv']
----------- Network-wide number of csv files -> 
 ----------- train : 15 ['/kaggle/input/training/ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-32-labeled.csv', '/kaggle/input/training/ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-15-labeled.csv', '/kaggle/input/training/ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-30-labeled.csv', '/kag

In [15]:
def _init_weights( module):
    ## for one layer apply Xavier Initialization
    if isinstance(module, nn.Linear):
        init.xavier_normal_(module.weight)
        if module.bias is not None:
            init.zeros_(module.bias)
    return module
# AutoEncoder (AE)
class AE(nn.Module):
    """
    Encoder: (89-64-32)
    Decoder: (32-64-89)
    """
    def __init__(self):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(89, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 89),
            nn.ReLU()
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon



In [15]:
from collections import Counter

def train_eval(model,train_dataloader,val_dataloader,test_dataloader,learning_rates= [5e-5,1e-5,1e-6,5e-6,1e-7],
               weight_decays=[1e-5,1e-4,1e-7],shuffle_files=True,num_epochs=50):
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    criterion = nn.MSELoss(reduction='sum').to(device)
    eval_criterion = nn.MSELoss(reduction='none').to(device)
    for lr, wd in itertools.product(learning_rates, weight_decays):
        if model._get_name()=="AdversarialAutoencoder":
            adversarial_criterion= nn.BCELoss(reduction="sum")
            optimizer_D = optim.SGD(model.discriminator.parameters(), lr=lr, weight_decay=wd)
            optimizer_G =  optim.SGD(list(model.encoder.parameters()) + list(model.decoder.parameters()), lr=lr, weight_decay=wd)
        else:
            AE_optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd)
        print(f"\n==================  lr={lr}, wd={wd} ==================")
        model.apply(_init_weights)
        for epoch in range(num_epochs):
            time_1 = time.time()
            model.train()
            train_loss = 0
            ## for AAE
            Discriminator_loss = 0
            if shuffle_files:
                sys_rand = SystemRandom()
                sys_rand.shuffle(AE_train_dataset.file_order_indices)
            for sequences, _ in train_dataloader:
                sequences=sequences.squeeze().to(device)
                if model._get_name()=="AdversarialAutoencoder":
                    target_ones= torch.ones(sequences.size(0), 1,device=device,dtype=torch.float)
                    target_zeros= torch.zeros(sequences.size(0), 1,device=device,dtype=torch.float)
                    random_latent = torch.randn(sequences.size(0), 2, device=device)
                    optimizer_G.zero_grad()
                    fake_z,decoded_seq = model(sequences)
                    G_loss = 0.001*adversarial_criterion(model.discriminator(fake_z),target_ones ) + 0.999*criterion(decoded_seq, sequences)
                    G_loss.backward()
                    optimizer_G.step()
                    # 2) discriminator loss
                    optimizer_D.zero_grad()
                    real_loss = adversarial_criterion(model.discriminator(random_latent), target_ones)
                    fake_loss = adversarial_criterion(model.discriminator(fake_z.detach()),  target_zeros)
                    D_loss = 0.5*(real_loss + fake_loss)
                    D_loss.backward()
                    optimizer_D.step()
                    train_loss+=G_loss.item()
                    Discriminator_loss+=D_loss.item()   
                else:
                    AE_optimizer.zero_grad()
                    if model._get_name()=="AE":
                        recon = model(sequences)
                        loss = criterion(recon, sequences) / sequences.size(0)
                    elif model._get_name()=="VAE" or model._get_name()=="GRUVAE":
                        recon, mu, logvar = model(sequences)
                        loss = vae_loss_function(recon, sequences, mu, logvar) /sequences.size(0)
                    loss.backward()
                    AE_optimizer.step()
                    train_loss += loss.item()
            print(f"Train : time {(time.time()-time_1):.2f} s",
            f"Epoch {epoch+1}")
            if model._get_name()=="AdversarialAutoencoder":
                print(f"Generator Loss: {train_loss / len(train_dataloader):.4f}",
                    f"Discriminator Loss: {Discriminator_loss / len(train_dataloader):.4f}")
            else:
                print(f"Train Loss: {train_loss / len(train_dataloader):.4f}")
            # Evaluate part
            if (epoch + 1) % 5 == 0:
                model.eval() 
                all_val_losses = []
                all_val_labels = []
                print(f"--- Running Evaluation for Epoch {epoch+1} lr ={lr} wd {wd} ---")
                with torch.no_grad():
                    for sequences, labels in val_dataloader:
                        sequences = sequences.squeeze().to(device)        
                        if model._get_name()=="AE":
                            recon = model(sequences)
                        elif model._get_name()=="VAE" or model._get_name()=="GRUVAE" :
                            recon, _, _ = model(sequences)
                        elif model._get_name()=="AdversarialAutoencoder":
                            _,recon= model(sequences)
                        val_loss = eval_criterion(recon, sequences)
                        if val_loss.dim() > 1:
                            intrusion_scores = val_loss
                        else:
                            intrusion_scores = val_loss.unsqueeze(dim=0)
                        intrusion_scores = intrusion_scores.sum(dim=1)
                        all_val_losses.extend(intrusion_scores.cpu().numpy())
                        all_val_labels.extend(labels.flatten().cpu().numpy())            
                threshold_1 = compute_threshold(all_val_losses,k=1)
                # threshold_2 = compute_threshold(all_val_losses,k=2)
                # threshold_3 = compute_threshold(all_val_losses,k=3)
                # threshold_10 = compute_threshold(all_val_losses,k=10)
                # threshold_ = compute_threshold(all_val_losses,k=100)

                # print(f"Computed Threshold: {threshold:.4f}")
                all_val_losses = np.array(all_val_losses).squeeze()  
                all_val_labels = np.array(all_val_labels).squeeze()  
                # If intrusion score > threshold, predict 1 (intrusion), else 0 (benign)
                # For FDR, get True Positives (TP) and False Positives (FP)
                
                predictions = (all_val_losses > threshold_1).astype(int)
                accuracy = accuracy_score(all_val_labels, predictions)
                print(f"Val: Accuracy: {accuracy:.4f}  ")
                model.eval() 
                all_test_losses = []
                all_test_labels = []
                with torch.no_grad():
                    for sequences, labels in test_dataloader:
                        sequences = sequences.squeeze().to(device)
                        labels = labels.squeeze().to(device)
                        if model._get_name()=="AE":
                            recon = model(sequences)
                        elif model._get_name()=="VAE"  or model._get_name()=="GRUVAE":
                            recon, mu, logvar = model(sequences)
                        elif model._get_name()=="AdversarialAutoencoder":
                            _,recon= model(sequences)
                        test_loss = eval_criterion(recon, sequences)
                        if test_loss.dim() > 1:
                            intrusion_scores = test_loss
                        else:
                            intrusion_scores = test_loss.unsqueeze(dim=0)
                        intrusion_scores = intrusion_scores.sum(dim=1).squeeze()
                        ### remove labels =1 (don't consider replay attack)
                        mask = labels != 1

                        filtered_scores = intrusion_scores[mask]
                        filtered_labels = labels[mask]
                
                        # Move to CPU and convert to numpy
                        all_test_losses.extend(filtered_scores.cpu().numpy())
                        all_test_labels.extend(filtered_labels.cpu().numpy())

                all_test_losses = np.array(all_test_losses)
                all_test_labels = np.array(all_test_labels)
                # for threshold in {threshold_1,threshold_2,threshold_3,threshold_10,threshold_100}:
                for threshold in {threshold_1}:

                    print("---thr:",threshold)
                    predictions = (all_test_losses > threshold).astype(int)
                    binary_test_labels = (all_test_labels != 0).astype(int)
                    # --- Start of new code ---

                    # Find the indices where the prediction was incorrect
                    misclassified_indices = np.where(binary_test_labels != predictions)[0]

                    # Get the original labels for those misclassified instances
                    misclassified_original_labels = all_test_labels[misclassified_indices]

                    # # Get the predicted labels for the misclassified instances
                    # misclassified_predictions = predictions[misclassified_indices]

                    # To get a summary count of which labels were misclassified
                    print(Counter(predictions),Counter(binary_test_labels))
                    print(f"Counts of  labels: {dict(sorted(Counter(all_test_labels).items()))}")
                    print(f"Counts of misclassified original labels: {dict(sorted(Counter(misclassified_original_labels).items()))}")

                    # # For a more detailed view, you can see the original label and what it was predicted as
                    # print("\n--- Detailed Misclassifications (Original Label -> Predicted Binary) ---")
                    # for i in range(len(misclassified_original_labels)):
                    #     original_label = misclassified_original_labels[i]
                    #     predicted_label = misclassified_predictions[i]
                    #     print(f"Original Label: {original_label} -> Predicted as: {predicted_label}")

                    accuracy = accuracy_score(binary_test_labels, predictions)
                    f1 = f1_score(binary_test_labels, predictions, zero_division=0)
                    recall = recall_score(binary_test_labels, predictions,zero_division=0)
                    _, fp, _, tp = confusion_matrix(binary_test_labels, predictions, labels=[0, 1]).ravel()
                    # FDR = FP / (FP + TP) 
                    if (fp + tp) == 0:
                        fdr = 0.0 
                    else:
                        fdr = fp / (fp + tp)
                    print(f"Test : Accuracy: {accuracy:.4f} Recall : {recall:.4f} FDR: {fdr:.4f}  F1-score: {f1:.4f}  ")

        # thresholds = compute_threshold(all_val_losses,k=3)
        #         # print(f"Computed Threshold: {threshold:.4f}")
        #         all_val_losses = np.array(all_val_losses).squeeze()  
        #         all_val_labels = np.array(all_val_labels).squeeze()  
        #         # If intrusion score > threshold, predict 1 (intrusion), else 0 (benign)
        #         # For FDR, get True Positives (TP) and False Positives (FP)
        #         for key,thresh_value in thresholds.items():
        #             print("*******",key,thresh_value,"threshold")
        #             predictions = (all_val_losses > thresh_value).astype(int)
        #             accuracy = accuracy_score(all_val_labels, predictions)
        #             print(f"Val: Accuracy: {accuracy:.4f}  ")
        #             model.eval() 
        #             all_test_losses = []
        #             all_test_labels = []
        #             with torch.no_grad():
        #                 for sequences, labels in test_dataloader:

In [28]:
loaded_scalers = load_scalers("/kaggle/working/FLBased-ICS-NIDS/fitted_scalers")
#standard_scalers

AE_train_dataset=ModbusFlowStream( 
    shuffle=True,chunk_size=20,batch_size=64,csv_files=val_files[:-3],scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=1,
    
)

AE_train_dataloader=DataLoader(AE_train_dataset,batch_size=1,shuffle=False,pin_memory=True, prefetch_factor=2,num_workers=4)

AE_val_dataloader=DataLoader(ModbusFlowStream( 
    shuffle=False,chunk_size=1,batch_size=64,csv_files=val_files[-3:],scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=1
),batch_size=1,shuffle=False,pin_memory=True)

AE_test_dataloader=DataLoader(ModbusFlowStream(shuffle=False,chunk_size=1,batch_size=64,csv_files=test_files[:],scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=1),batch_size=1,shuffle=False,pin_memory=True)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
AE_model = AE().to(device)
lr = 0.01
wd= 1e-4
shuffle_files =True
AE_optimizer = optim.SGD(AE_model.parameters(), lr=lr, weight_decay=wd)
criterion = nn.MSELoss(reduction='sum').to(device)
eval_criterion = nn.MSELoss(reduction='none').to(device)



Successfully loaded scalers for 'network-wide'


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [62]:
print(AE_train_dataset.scalers)

{'Protocol': MinMaxScaler(), 'Flow Duration': MinMaxScaler(), 'Total Fwd Packet': MinMaxScaler(), 'Total Bwd packets': MinMaxScaler(), 'Total Length of Fwd Packet': MinMaxScaler(), 'Total Length of Bwd Packet': MinMaxScaler(), 'Fwd Packet Length Max': MinMaxScaler(), 'Fwd Packet Length Min': MinMaxScaler(), 'Fwd Packet Length Mean': MinMaxScaler(), 'Fwd Packet Length Std': MinMaxScaler(), 'Bwd Packet Length Max': MinMaxScaler(), 'Bwd Packet Length Min': MinMaxScaler(), 'Bwd Packet Length Mean': MinMaxScaler(), 'Bwd Packet Length Std': MinMaxScaler(), 'Flow Bytes/s': MinMaxScaler(), 'Flow Packets/s': MinMaxScaler(), 'Flow IAT Mean': MinMaxScaler(), 'Flow IAT Std': MinMaxScaler(), 'Flow IAT Max': MinMaxScaler(), 'Flow IAT Min': MinMaxScaler(), 'Fwd IAT Total': MinMaxScaler(), 'Fwd IAT Mean': MinMaxScaler(), 'Fwd IAT Std': MinMaxScaler(), 'Fwd IAT Max': MinMaxScaler(), 'Fwd IAT Min': MinMaxScaler(), 'Bwd IAT Total': MinMaxScaler(), 'Bwd IAT Mean': MinMaxScaler(), 'Bwd IAT Std': MinMaxScal

In [29]:
train_eval(AE_model,AE_train_dataloader,AE_val_dataloader,AE_test_dataloader)



==================  lr=5e-05, wd=1e-05 ==================
Train : time 36.23 s Epoch 1
Train Loss: 3.5386
Train : time 33.80 s Epoch 2
Train Loss: 2.8849


KeyboardInterrupt: 

In [54]:
train_eval(AE_model,AE_train_dataloader,AE_val_dataloader,AE_test_dataloader)



==================  lr=5e-05, wd=1e-05 ==================
Train : time 115.05 s Epoch 1
Train Loss: 3.1154
--- Running Evaluation for Epoch 1 lr =5e-05 wd 1e-05 ---
----------- tensor(2.0512) tensor(0.0694)
[tensor(6.1216), tensor(6.1216), tensor(6.1216), tensor(6.1216), tensor(6.1216), tensor(6.1216), tensor(6.1216), tensor(6.1216), tensor(6.1216)]
Val: Accuracy: 0.9994  
---thr: 2.120652437210083
Counter({0: 362790, 1: 127374}) Counter({0: 383754, 1: 106410})
Counts of  labels: {0: 383754, 1: 105690, 2: 178, 3: 101, 4: 125, 5: 118, 6: 86, 7: 112}
Counts of misclassified original labels: {0: 37535, 1: 16464, 2: 33, 3: 40, 5: 1, 6: 32, 7: 1}
Test : Accuracy: 0.8896 Recall : 0.8443 FDR: 0.2947  F1-score: 0.7686  
Train : time 111.38 s Epoch 2
Train Loss: 2.1930
--- Running Evaluation for Epoch 2 lr =5e-05 wd 1e-05 ---
----------- tensor(2.0472) tensor(0.0684)
[tensor(6.0578), tensor(6.0578), tensor(6.0578), tensor(6.0578), tensor(6.0578), tensor(6.0578), tensor(6.0578), tensor(6.0578),

KeyboardInterrupt: 

In [24]:
train_eval(AE_model,AE_train_dataloader,AE_val_dataloader,AE_test_dataloader)



================== Evaluate lr=0.01, wd=0.0001 ==================
Train : time 96.0562 Epoch 0
Train Loss: 0.8941

--- Running Evaluation for Epoch 1 lr =0.01 wd 0.0001 ---
Computed Threshold: 0.0179
Val: Accuracy: 0.9124  
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
Counter({0: 364401, 1: 125763}) Counter({0: 383754, 1: 106410})
Counts of  labels: Counter({0: 383754, 1: 105690, 2: 178, 4: 125, 5: 118, 7: 112, 3: 101, 6: 86})
Counts of misclassified labels: Counter({0: 37095, 1: 17516, 7: 112, 3: 43, 6: 34, 2: 33, 4: 3, 5: 1})
Test : Accuracy: 0.8881 Recall : 0.8333 FDR: 0.2950  F1-score: 0.7638  
Train : time 95.1892 Epoch 1
Train Loss: 0.8722

--- Running Evaluation for Epoch 2 lr =0.01 wd 0.0001 ---
Computed Threshold: 0.0179
Val: Accuracy: 0.9117  
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
Counter({0: 364596, 1: 125568}) Counter({0: 383754, 1: 106410})
Counts of  labels: Counter({0: 383754, 1: 105690, 2: 178, 4: 125, 5: 118, 7: 112, 3: 101, 6: 86})
Counts of misclassified labels: Counter({0: 36

KeyboardInterrupt: 

In [55]:

# Variational AutoEncoder (VAE)
class VAE(nn.Module):
    """
    Encoder: (89-64-64-32 for mu and log_var)
    Decoder: (32-64-64-89)
    return x_recon, mu, logvar
    """
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(89, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(64, 32)
        self.fc_logvar = nn.Linear(64, 32)
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 89),
            nn.ReLU()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        return x_recon, mu, logvar



In [ ]:
VAE_model = VAE().to(device=device)
train_eval(VAE_model,AE_train_dataloader,AE_val_dataloader,AE_test_dataloader)



==================  lr=5e-05, wd=1e-05 ==================
Train : time 161.09 s Epoch 1
Train Loss: 3.6338
--- Running Evaluation for Epoch 1 lr =5e-05 wd 1e-05 ---
----------- tensor(1.4726) tensor(0.3744)
[tensor(10.1306), tensor(10.1807), tensor(10.1918), tensor(10.3417), tensor(10.8254), tensor(11.2557), tensor(11.2946), tensor(11.5391), tensor(12.5028)]
Val: Accuracy: 0.8675  
---thr: 1.8469618558883667
Counter({0: 391335, 1: 98829}) Counter({0: 383754, 1: 106410})
Counts of  labels: {0: 383754, 1: 105690, 2: 178, 3: 101, 4: 125, 5: 118, 6: 86, 7: 112}
Counts of misclassified original labels: {0: 61252, 1: 68332, 2: 116, 3: 73, 4: 45, 5: 104, 6: 64, 7: 99}
Test : Accuracy: 0.7346 Recall : 0.3531 FDR: 0.6198  F1-score: 0.3662  
Train : time 154.61 s Epoch 2
Train Loss: 2.8694
--- Running Evaluation for Epoch 2 lr =5e-05 wd 1e-05 ---
----------- tensor(1.3536) tensor(0.3585)
[tensor(9.8622), tensor(10.1911), tensor(10.2131), tensor(10.4283), tensor(10.4542), tensor(10.6005), tensor

In [ ]:

VAE_model = VAE().to(device=device)
train_eval(VAE_model,AE_train_dataloader,AE_val_dataloader,AE_test_dataloader)


# for epoch in range(3):
#     time_1 = time.time()
#     train_loss = 0
#     AE_model.train()
#     if shuffle_files:
#         sys_rand = SystemRandom()
#         sys_rand.shuffle(AE_dataset.file_order_indices)
#     for sequences, _ in AE_dataloader:
#         sequences = sequences.squeeze().to(device)
#         VAE_optimizer.zero_grad()
#         recon, mu, logvar = VAE_model(sequences)
#         loss = vae_loss_function(recon, sequences, mu, logvar)
#         loss.backward()
#         VAE_optimizer.step()
#         train_loss += loss.item()
#     print("time",time.time()-time_1,f"Epoch {epoch}, Train Loss: {train_loss / len(AE_dataloader)}")



================== Evaluate lr=0.01, wd=0.0001 ==================
Train : time 141.5818 Epoch 0 Train Loss: 97.8765

--- Running Evaluation for Epoch 1 lr =0.01 wd 0.0001 ---
Computed Threshold: 0.0206
Val: Accuracy: 0.9378  
Test : Accuracy: 0.8535 Recall : 0.8524 FDR: 0.3822  F1-score: 0.7164  
Train : time 139.1011 Epoch 1 Train Loss: 93.5440

--- Running Evaluation for Epoch 2 lr =0.01 wd 0.0001 ---
Computed Threshold: 0.0382
Val: Accuracy: 0.8970  
Test : Accuracy: 0.8366 Recall : 0.8565 FDR: 0.4156  F1-score: 0.6948  

================== Evaluate lr=0.01, wd=1e-05 ==================
Train : time 135.4118 Epoch 0 Train Loss: 102.8192

--- Running Evaluation for Epoch 1 lr =0.01 wd 1e-05 ---
Computed Threshold: 0.0314
Val: Accuracy: 0.8984  
Test : Accuracy: 0.2174 Recall : 0.9999 FDR: 0.7829  F1-score: 0.3568  
Train : time 134.7249 Epoch 1 Train Loss: 102.6428

--- Running Evaluation for Epoch 2 lr =0.01 wd 1e-05 ---
Computed Threshold: 0.0322
Val: Accuracy: 0.8927  
Test : Accu

In [29]:

class AAE_Encoder(nn.Module):
    def __init__(self):
        """
        Encoder(Generator):(89-16-4-2)
        """
        super(AAE_Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(89, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 4),
            nn.LeakyReLU(0.2),
            nn.Linear(4, 2))
    def forward(self, x):
        return self.encoder(x)
class AAE_Decoder(nn.Module):
    def __init__(self):
        super(AAE_Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 89),
            nn.LeakyReLU()
        )
    def forward(self, x):
        return self.decoder(x)
class AAE_Discriminator(nn.Module):
    def __init__(self):
        super(AAE_Discriminator, self).__init__()
        # corrected to 2-16-4-1
        self.discriminator = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 1), 
            nn.Sigmoid()
        )    
    def forward(self, x):
        return self.discriminator(x)
 
class AdversarialAutoencoder(nn.Module):
    def __init__(self):
        super(AdversarialAutoencoder, self).__init__()
        self.encoder = AAE_Encoder()
        self.decoder = AAE_Decoder()
        self.discriminator = AAE_Discriminator()
    def forward(self, x):
        fake_z = self.encoder(x)
        x_recon = self.decoder(fake_z)
        return fake_z,x_recon


In [28]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
aae_model = AdversarialAutoencoder().to(device)
device = next(aae_model.parameters()).device
print(f"Model device (Method 1): {device}")
print(next(aae_model.decoder.parameters()).device)
print(aae_model._get_name())
train_eval(aae_model,AE_train_dataloader,AE_val_dataloader,AE_test_dataloader)

# for epoch in range(num_epochs):
#     # aae_encoder.train()
#     # aae_decoder.train()
#     # aae_discriminator.train()
#     aae_model.train()
#     if shuffle_files:
#         sys_rand = SystemRandom()
#         sys_rand.shuffle(AE_dataset.file_order_indices)
#     for sequences,_ in AE_dataloader:
#         sequences=sequences.squeeze().to(device)
#         # 1) reconstruction + generator loss
#         optimizer_G.zero_grad()
#         fake_z = aae_encoder(sequences)
#         decoded_seq = aae_decoder(fake_z)
#         G_loss = 0.001*adversarial_loss(aae_discriminator(fake_z),  torch.ones(sequences.size(0), 2,device=device)) + 0.999*reconstruction_loss(decoded_seq, sequences)
#         G_loss.backward()
#         optimizer_G.step()
#         # 2) discriminator loss
#         optimizer_D.zero_grad()
#         real_loss = adversarial_loss(aae_discriminator(torch.randn(sequences.size(0), 2, device=device)),  torch.ones(sequences.size(0), 2,device=device))
#         fake_loss = adversarial_loss(aae_discriminator(fake_z.detach()),  torch.zeros(sequences.size(0), 2,device=device))
#         D_loss = 0.5*(real_loss + fake_loss)
#         D_loss.backward()
#         optimizer_D.step()
#     # print loss
#     print(
#             "[Epoch %d/%d] [G loss: %f] [D loss: %f]"
#             % (epoch, num_epochs, G_loss.item(), D_loss.item())
#          )

Model device (Method 1): cuda:0
cuda:0
AdversarialAutoencoder

================== Evaluate lr=0.01, wd=0.0001 ==================
Train : time 253.4021 Epoch 0
Generator Loss: 0.8527 Discriminator Loss: 61.5562

--- Running Evaluation for Epoch 1 lr =0.01 wd 0.0001 ---
Computed Threshold: 0.0011
Val: Accuracy: 0.9964  
Test : Accuracy: 0.8894 Recall : 0.8443 FDR: 0.2954  F1-score: 0.7681  
Train : time 228.5464 Epoch 1
Generator Loss: 0.5067 Discriminator Loss: 60.1043

--- Running Evaluation for Epoch 2 lr =0.01 wd 0.0001 ---
Computed Threshold: 0.0011
Val: Accuracy: 0.9939  
Test : Accuracy: 0.8894 Recall : 0.8443 FDR: 0.2954  F1-score: 0.7682  

================== Evaluate lr=0.01, wd=1e-05 ==================
Train : time 210.8485 Epoch 0
Generator Loss: 1.1983 Discriminator Loss: 299.7112

--- Running Evaluation for Epoch 1 lr =0.01 wd 1e-05 ---
Computed Threshold: 0.0014
Val: Accuracy: 0.9957  
Test : Accuracy: 0.8894 Recall : 0.8443 FDR: 0.2954  F1-score: 0.7681  
Train : time 207

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
aae_model = AdversarialAutoencoder().to(device)
device = next(aae_model.parameters()).device
print(f"Model device (Method 1): {device}")
print(next(aae_model.decoder.parameters()).device)
print(aae_model._get_name())
train_eval(aae_model,AE_train_dataloader,AE_val_dataloader,AE_test_dataloader)


In [35]:
# GRU-VAE
class GRUVAE(nn.Module):
    """
    Gated Recurrent Unit : num_layers=2, hidden_size=256, dropout=0.01,window size (seq_len)= 40
    """
    def __init__(self, input_dim=89, hidden_dim=256, latent_dim=32, num_layers=2, dropout=0.01):
        super(GRUVAE, self).__init__()
        self.encoder_gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        self.fc_z_to_hidden = nn.Linear(latent_dim, hidden_dim)
        self.decoder_gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(hidden_dim, input_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # x shape: [batch_size, seq_len, input_dim=89]
        _, hidden = self.encoder_gru(x) 
        h = hidden[-1]  # [batch_size, hidden_dim]
        mu = self.fc_mu(h)  
        logvar = self.fc_logvar(h)  
        z = self.reparameterize(mu, logvar)  # [batch_size, latent_dim]
        # repeat and feed latent z as input trick
        h0 = self.fc_z_to_hidden(z).unsqueeze(0).repeat(self.encoder_gru.num_layers, 1, 1)  # [num_layers, batch_size, hidden_dim]
        # Initialize decoder input with zeros 
        decoder_input = torch.zeros_like(x)
        output, _ = self.decoder_gru(decoder_input, h0)  # [batch_size, seq_len, hidden_dim]
        x_recon = self.fc_out(output)  # [batch_size, seq_len, input_dim]
        return x_recon, mu, logvar


In [39]:
loaded_scalers = load_scalers("fitted_scalers")
RNN_train_dataset=ModbusFlowStream( 
    shuffle=False,chunk_size=1,batch_size=64,csv_files=val_files[:-3],
    scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=30
)
RNN_train_dataloadder=DataLoader(RNN_train_dataset,batch_size=1,shuffle=False)
RNN_test_dataset=ModbusFlowStream( 
    shuffle=False,chunk_size=1,batch_size=64,csv_files=val_files[-3:],
    scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=30
)
RNN_test_dataset=ModbusFlowStream( 
    shuffle=False,chunk_size=1,batch_size=64,csv_files=test_files,
    scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=30
)
RNN_test_dataloadder=DataLoader(RNN_test_dataset,batch_size=1,shuffle=False)

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
GRU_VAE_model = GRUVAE().to(device)
lr = 0.01
wd= 1e-4
shuffle_files =True
GRU_VAE_optimizer = optim.SGD(GRU_VAE_model.parameters(), lr=lr, weight_decay=wd)



Successfully loaded scalers for 'network-wide'


In [42]:
train_eval(GRU_VAE_model,RNN_train_dataloadder,RNN_test_dataloader,RNN_test_dataloader)



==================  lr=5e-05, wd=1e-05 ==================
Train : time 219.29 s Epoch 1
Train Loss: 4.8276
--- Running Evaluation for Epoch 1 lr =5e-05 wd 1e-05 ---


/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([9, 30, 89])) that is different to the input size (torch.Size([64, 30, 89])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (64) must match the size of tensor b (9) at non-singleton dimension 0

In [6]:
for epoch in range(3):
    time_1 = time.time()
    train_loss = 0
    GRU_VAE_model.train()
    if shuffle_files:
        sys_rand = SystemRandom()
        sys_rand.shuffle(RNN_dataset.file_order_indices)
    for sequences, _ in RNN_dataloadder:
        sequences = sequences.squeeze().to(device)
        GRU_VAE_optimizer.zero_grad()
        recon, mu, logvar = GRU_VAE_model(sequences)
        loss = vae_loss_function(recon, sequences, mu, logvar)/sequences.size(0)
        loss.backward()
        GRU_VAE_optimizer.step()
        train_loss += loss.item()
    print("time",time.time()-time_1,f"Epoch {epoch}, Train Loss: {train_loss/len(RNN_dataloadder)}")


time 103.30588173866272 Epoch 0, Train Loss: 127700652.60516566
time 95.60702395439148 Epoch 1, Train Loss: 70232281.69612122
time 94.19640827178955 Epoch 2, Train Loss: 3457994.1813964844
